# Can Congress and Twitter data be linked?

>How to use Congress API: https://github.com/eyeseast/propublica-congress/blob/master/docs/api.rst

In [14]:
import tweepy
import pandas as pd

## Set up Congress API: Senate

In [17]:
from congress import Congress

congress = Congress('qa1qfTzGinGGLcNCbinp2OTlI2SvzBe4KFgxYobm')

# get member by bioguide ID
senate = congress.members.filter('senate')

In [42]:
#access members in dict
sen_members = senate[0]['members']

#save in df
df_senate = pd.DataFrame(sen_members)
df_senate.columns #twitter_account gives twitter name

Index(['api_uri', 'contact_form', 'crp_id', 'cspan_id', 'date_of_birth',
       'dw_nominate', 'facebook_account', 'fax', 'fec_candidate_id',
       'first_name', 'gender', 'google_entity_id', 'govtrack_id', 'icpsr_id',
       'id', 'ideal_point', 'in_office', 'last_name', 'leadership_role',
       'lis_id', 'middle_name', 'missed_votes', 'missed_votes_pct',
       'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url',
       'senate_class', 'seniority', 'short_title', 'state', 'state_rank',
       'suffix', 'title', 'total_present', 'total_votes', 'twitter_account',
       'url', 'votes_with_party_pct', 'votesmart_id', 'youtube_account'],
      dtype='object')

In [54]:
total = df_senate.shape[0]

In [57]:
print ("Share of Senators without Twitter account:", (df_senate.twitter_account.isnull().sum()/total)*100,'\n'
      "Share of Senators without YouTube account:", (df_senate.youtube_account.isnull().sum()/total)*100
      )

Share of Senators without Twitter account: 3.8461538461538463 
Share of Senators without YouTube account: 15.384615384615385


>Good News: only 3% do not have a twitter account

## Set up Tweepy

In [58]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

In [72]:
consumer_key= 'AKEpYmXTTBDu6lePXsBfOz3zy'
consumer_secret = 'h6Kz7HPfmGVCVqsKeck6kSwOsIxnIHeMpEy6NOUkfSYHjF8NMJ'
access_token="991768520546488321-OvBfDZg6YEZpo3oWnabsOj5rhRBCNvY"
access_token_secret="wLJUj2hDB0wE4A4x2TgNnesEqFmqHbyRlNvMorWjcoMfw"
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

In [85]:
#save account names in list an skip 'None'
account_list = [i for i in df_senate['twitter_account'] if not i == None] 
account_list[:3]

['SenAlexander', 'SenatorBaldwin', 'SenJohnBarrasso']

In [103]:
for target in account_list[:3]: #just a subset
    print ('%s' %target)
    item = auth_api.get_user(target)
    tweets = item.statuses_count
    account_created_date = item.created_at
    delta = datetime.utcnow() - account_created_date
    account_age_days = delta.days
    print("Account age (in days): " + str(account_age_days))
    
    if account_age_days > 0:
        print("Average tweets per day: " + "%.2f \n"%(float(tweets)/float(account_age_days)))

SenAlexander
Account age (in days): 3143
Average tweets per day: 1.40 

SenatorBaldwin
Account age (in days): 1939
Average tweets per day: 5.96 

SenJohnBarrasso
Account age (in days): 2758
Average tweets per day: 1.37 



#### How actively are they tweeting?